In [ ]:
with open('/content/drive/MyDrive/StableDiffusion_Emote_Checkpoints/test_file2.txt','w') as f:
  f.write("Mounted file wokring!")

In [ ]:
!pip install diffusers transformers accelerate datasets safetensors wandb

In [ ]:
!pip uninstall -y diffusers
!pip install git+https://github.com/huggingface/diffusers

Found existing installation: diffusers 0.35.2
Uninstalling diffusers-0.35.2:
  Successfully uninstalled diffusers-0.35.2
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-d9sq21hr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-d9sq21hr
  Resolved https://github.com/huggingface/diffusers to commit 6f1042e36cd588a7b66498f45c3bb7085e4fa395
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-py3-none-any.whl size=4475017 sha256=c8ff6fe47d3b42237c46a88c7a293c06fb27ea23c82301f89c0dcee97f09d33c
  Stored in directory: /tmp/pip-ephem-wheel-cache-rkpk8ska/wheels/90/d4/44/a58bc00fb405fefb633b0d9d2307f6e3aec6cc1775d82555d3
Successfully built diffusers


In [ ]:
import diffusers
print(diffusers.__version__)

0.35.2


In [ ]:
import torch
print("Torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

Torch: 2.9.0+cu126
CUDA: 12.6
CUDA available: True


In [ ]:
from huggingface_hub import notebook_login, whoami
notebook_login()

In [ ]:
whoami()

{'type': 'user',
 'id': '66a797b589b3e71262932d0d',
 'name': 'SurAyush',
 'fullname': 'Ayush Sur',
 'email': 'ayushsur26@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'billingMode': 'postpaid',
 'periodEnd': None,
 'isPro': False,
 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/noauth/RZJZW_w0wdVoOmQY250lR.png',
 'orgs': [{'type': 'org',
   'id': '681b0cb0dba891d54be0773d',
   'name': 'mcp-course',
   'fullname': 'Hugging Face MCP Course',
   'email': None,
   'canPay': False,
   'billingMode': 'postpaid',
   'periodEnd': None,
   'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/62d648291fa3e4e7ae3fa6e8/itgTDqMrnvgNfJZJ4YmCt.png',
   'roleInOrg': 'read',
   'isEnterprise': False}],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'st_df1.5',
   'role': 'write',
   'createdAt': '2025-11-04T13:00:07.382Z'}}}

In [ ]:
!wandb login

wandb: Currently logged in as: ayush_sur (ayush_sur-national-institute-of-technlogy-kurukshetra) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# configuration.py

from dataclasses import dataclass

@dataclass
class TrainingConfig:
    pretrained_model_name_or_path: str = "stable-diffusion-v1-5/stable-diffusion-v1-5"
    dataset_name: str = "valhalla/emoji-dataset"
    output_dir: str = "/content/drive/MyDrive/StableDiffusion_Emote_Checkpoints/sddata/finetune_lora"
    mixed_precision: str = "bf16"
    resume_from_checkpoint: str = "checkpoint-4500"
    dataloader_num_workers: int = 2
    resolution: int = 256
    train_batch_size: int = 8
    gradient_accumulation_steps: int = 1
    max_train_steps: int = 7500
    learning_rate: int = 1e-04
    max_grad_norm: int = 1
    lr_scheduler: str = "cosine"
    lr_warmup_steps: int = 0
    push_to_hub: bool = True
    hub_model_id: str = "StableEmote-lora"
    report_to: str= "wandb"
    logging_dir: str ="logs"
    checkpointing_steps: int = 500
    validation_prompt: str ="An emoji of robot face"
    num_validation_images: int = 4
    validation_epochs: int = 1
    image_interpolation_mode: str = "lanczos"
    center_crop: bool = False
    random_flip: bool = False
    learning_rate: float = 1e-4
    adam_beta1: float = 0.9
    adam_beta2: float = 0.999
    adam_weight_decay: float = 1e-2
    adam_epsilon: float = 1e-8
    rank: int = 4
    enable_xformers_memory_efficient_attention: bool = False
    gradient_checkpointing: bool = False
    seed: int = 42
    hub_token: str=""



In [ ]:
#logger_setup.py

# W&B logging setup
import logging
import wandb
import torch
from accelerate.logging import get_logger
import numpy as np

def log_validation(logger, pipeline,config,accelerator,epoch,is_final_validation=False):

    logger.info(f"Running validation... \n Generating {config.num_validation_images} images with prompt:"f" {config.validation_prompt}.")

    pipeline = pipeline.to(accelerator.device)
    pipeline.set_progress_bar_config(disable=True)
    generator = torch.Generator(device=accelerator.device)
    if config.seed is not None:
        generator = generator.manual_seed(config.seed)

    images = []
    autocast_ctx = torch.autocast(accelerator.device.type)

    with autocast_ctx:
        for _ in range(config.num_validation_images):
            images.append(pipeline(config.validation_prompt, num_inference_steps=40, generator=generator).images[0])

    for tracker in accelerator.trackers:
        phase_name = "test" if is_final_validation else "validation"
        if tracker.name == "tensorboard":
            np_images = np.stack([np.asarray(img) for img in images])
            tracker.writer.add_images(phase_name, np_images, epoch, dataformats="NHWC")

        if tracker.name == "wandb":
            tracker.log({phase_name: [wandb.Image(image, caption=f"{i}: {config.validation_prompt}") for i, image in enumerate(images)]})

    return images

def log_init(logger, len_train_dataset, config, total_batch_size):

    logger.info("***** Training *****")
    logger.info(f"  Num examples = {len_train_dataset}")
    logger.info(f"  Num Epochs = {config.num_train_epochs}")
    logger.info(f"  Instantaneous batch size per device = {config.train_batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {config.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {config.max_train_steps}")



In [ ]:
# save_model.py

import os
from diffusers.utils.hub_utils import load_or_create_model_card, populate_model_card


def save_model_card(repo_id: str, base_model: str = None, repo_folder: str = None):

    model_description = f"""
        # LoRA text2image fine-tuning of stable-diffusion-1.5
        These are LoRA adaption weights for stable-diffusion-1.5. The weights were fine-tuned on the valhalla/emoji-dataset dataset.
        Refer to the GitHub Repo for more information: https://github.com/SurAyush/StableEmote
    """

    model_card = load_or_create_model_card(
        repo_id_or_path=repo_id,
        from_training=True,
        base_model=base_model,
        model_description=model_description,
        inference=True,
    )

    tags = ["text-to-image","diffusers","stable-diffusion","lora"]

    model_card = populate_model_card(model_card, tags=tags)

    # Save the model card to the repo folder
    model_card.save(os.path.join(repo_folder, "README.md"))



In [ ]:
# compatible with both older and new version of diffusers
try:
    from diffusers.loaders.lora_conversion_utils import convert_unet_state_dict_to_peft
except ImportError:
    try:
        from diffusers.loaders.peft import convert_unet_state_dict_to_peft
    except ImportError:
        from diffusers.loaders.lora import convert_unet_state_dict_to_peft

import logging
import math
import os
from contextlib import nullcontext
from pathlib import Path

import datasets
import torch
import torch.nn.functional as F
import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import ProjectConfiguration, set_seed
from datasets import load_dataset
from huggingface_hub import create_repo, upload_folder
from packaging import version
from peft import LoraConfig
from peft.utils import get_peft_model_state_dict
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer

import diffusers
from diffusers import AutoencoderKL, DDPMScheduler, DiffusionPipeline, StableDiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.training_utils import cast_training_params
from diffusers.utils import check_min_version, convert_state_dict_to_diffusers
from diffusers.utils.import_utils import is_xformers_available
from diffusers.utils.torch_utils import is_compiled_module



from peft.utils import get_peft_model_state_dict, set_peft_model_state_dict

logger = get_logger(__name__, log_level="INFO")

# Will error if the minimal version of diffusers necessary is not installed
check_min_version("0.36.0.dev0")


def train():
    config = TrainingConfig()

    logging_dir = Path(config.output_dir, config.logging_dir)

    accelerator_project_config = ProjectConfiguration(project_dir=config.output_dir, logging_dir=logging_dir)

    accelerator = Accelerator(
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        mixed_precision=config.mixed_precision,
        log_with=config.report_to,
        project_config=accelerator_project_config,
    )

    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state, main_process_only=False)

    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_warning()
        diffusers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()
        diffusers.utils.logging.set_verbosity_error()


    set_seed(config.seed if config.seed is not None else 42)

    # Handle the repository creation
    if accelerator.is_main_process:

        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)

        if config.push_to_hub:
            repo_id = create_repo(
                repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True, token=config.hub_token
            ).repo_id

    # Load scheduler, tokenizer and models.
    noise_scheduler = DDPMScheduler.from_pretrained(config.pretrained_model_name_or_path, subfolder="scheduler")
    tokenizer = CLIPTokenizer.from_pretrained(config.pretrained_model_name_or_path, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(config.pretrained_model_name_or_path, subfolder="text_encoder")
    vae = AutoencoderKL.from_pretrained(config.pretrained_model_name_or_path, subfolder="vae")
    unet = UNet2DConditionModel.from_pretrained(config.pretrained_model_name_or_path, subfolder="unet")

    # freeze parameters of models: LoRA
    unet.requires_grad_(False)
    vae.requires_grad_(False)
    text_encoder.requires_grad_(False)

    # For mixed precision training we cast all non-trainable weights (vae, non-lora text_encoder and non-lora unet) to half-precision
    weight_dtype = torch.float32
    if accelerator.mixed_precision == "fp16":
        weight_dtype = torch.float16
    elif accelerator.mixed_precision == "bf16":
        weight_dtype = torch.bfloat16

    unet_lora_config = LoraConfig(
        r=config.rank,
        lora_alpha=config.rank,
        init_lora_weights="gaussian",
        target_modules=["to_k", "to_q", "to_v", "to_out.0"],
    )

    # Move unet, vae and text_encoder to device and cast to weight_dtype
    unet.to(accelerator.device, dtype=weight_dtype)
    vae.to(accelerator.device, dtype=weight_dtype)
    text_encoder.to(accelerator.device, dtype=weight_dtype)

    unet.add_adapter(unet_lora_config)
    if config.mixed_precision == "fp16":
        cast_training_params(unet, dtype=torch.float32)

    # xformer configuration for faster training
    if config.enable_xformers_memory_efficient_attention:
        if is_xformers_available():
            import xformers

            xformers_version = version.parse(xformers.__version__)
            if xformers_version == version.parse("0.0.16"):
                logger.warning(
                    "xFormers 0.0.16 cannot be used for training in some GPUs. If you observe problems during training, please update xFormers to at least 0.0.17. See https://huggingface.co/docs/diffusers/main/en/optimization/xformers for more details."
                )
            unet.enable_xformers_memory_efficient_attention()
        else:
            raise ValueError("xformers is not available. Make sure it is installed correctly")


    lora_layers = filter(lambda p: p.requires_grad, unet.parameters())

    # saves memory but slower training
    if config.gradient_checkpointing:
        unet.enable_gradient_checkpointing()

    optimizer_class = torch.optim.AdamW

    optimizer = optimizer_class(
        lora_layers,
        lr=config.learning_rate,
        betas=(config.adam_beta1, config.adam_beta2),
        weight_decay=config.adam_weight_decay,
        eps=config.adam_epsilon,
    )

    dataset = load_dataset(config.dataset_name)

    dataset_columns = ("image", "text")

    image_column = dataset_columns[0]
    caption_column = dataset_columns[1]


    # Get the specified interpolation method from the config
    interpolation = getattr(transforms.InterpolationMode, config.image_interpolation_mode.upper(), None)
    if interpolation is None:
        raise ValueError(f"Unsupported interpolation mode {config.image_interpolation_mode}.")

    # Data preprocessing transformations
    train_transforms = transforms.Compose(
        [
            transforms.Resize(config.resolution, interpolation=interpolation),  # Use dynamic interpolation method
            transforms.CenterCrop(config.resolution) if config.center_crop else transforms.RandomCrop(config.resolution),
            transforms.RandomHorizontalFlip() if config.random_flip else transforms.Lambda(lambda x: x),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ]
    )

    def unwrap_model(model):
        model = accelerator.unwrap_model(model)
        model = model._orig_mod if is_compiled_module(model) else model
        return model

    def tokenize_captions(examples):
        captions = []
        for caption in examples[caption_column]:
            if isinstance(caption, str):
                captions.append("An emoji of " + caption)     # prefixing captions with "An emoji of "
            else:
                raise ValueError(
                    f"Caption column `{caption_column}` should contain strings"
                )

        inputs = tokenizer(
            captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        return inputs.input_ids

    def preprocess_train(examples):
        images = [image.convert("RGB") for image in examples[image_column]]
        examples["pixel_values"] = [train_transforms(image) for image in images]
        examples["input_ids"] = tokenize_captions(examples)
        return examples

    def collate_fn(examples):
        pixel_values = torch.stack([example["pixel_values"] for example in examples])
        pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
        input_ids = torch.stack([example["input_ids"] for example in examples])
        return {"pixel_values": pixel_values, "input_ids": input_ids}


    with accelerator.main_process_first():
        # Set the training transforms
        train_dataset = dataset["train"].with_transform(preprocess_train)

    # DataLoaders creation:
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=collate_fn,
        batch_size=config.train_batch_size,
        num_workers=config.dataloader_num_workers,
    )

    def save_model_hook(models, weights, output_dir):
        if accelerator.is_main_process:
            unet_lora_layers_to_save = None

            for model in models:
                if isinstance(model, type(unwrap_model(unet))):
                    unet_lora_layers_to_save = get_peft_model_state_dict(model)
                else:
                    raise ValueError(f"Unexpected save model: {model.__class__}")

                # make sure to pop weight so that corresponding model is not saved again
                weights.pop()

            StableDiffusionPipeline.save_lora_weights(
                save_directory=output_dir,
                unet_lora_layers=unet_lora_layers_to_save,
                safe_serialization=True,
            )

    def load_model_hook(models, input_dir):
        unet_ = None

        while len(models) > 0:
            model = models.pop()
            if isinstance(model, type(unwrap_model(unet))):
                unet_ = model
            else:
                raise ValueError(f"unexpected save model: {model.__class__}")

        lora_state_dict, network_alphas = StableDiffusionPipeline.lora_state_dict(input_dir)

        unet_state_dict = {
            f'{k.replace("unet.", "")}': v for k, v in lora_state_dict.items() if k.startswith("unet.")
        }
        unet_state_dict = convert_unet_state_dict_to_peft(unet_state_dict)
        incompatible_keys = set_peft_model_state_dict(unet_, unet_state_dict, adapter_name="default")

        if incompatible_keys is not None:
            # check only for unexpected keys
            unexpected_keys = getattr(incompatible_keys, "unexpected_keys", None)
            if unexpected_keys:
                logger.warning(
                    f"Loading adapter weights from state_dict led to unexpected keys not found in the model: "
                    f" {unexpected_keys}. "
                )

        # Make sure the trainable params are in float32
        if config.mixed_precision in ["fp16", "bf16"]:
            cast_training_params([unet_], dtype=torch.float32)


    num_warmup_steps_for_scheduler = config.lr_warmup_steps * accelerator.num_processes
    num_training_steps_for_scheduler = config.max_train_steps * accelerator.num_processes

    lr_scheduler = get_scheduler(
        config.lr_scheduler,
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps_for_scheduler,
        num_training_steps=num_training_steps_for_scheduler,
    )

    # Prepare everything with accelerator.
    unet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        unet, optimizer, train_dataloader, lr_scheduler
    )

    accelerator.register_save_state_pre_hook(save_model_hook)
    accelerator.register_load_state_pre_hook(load_model_hook)

    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / config.gradient_accumulation_steps)
    config.num_train_epochs = math.ceil(config.max_train_steps / num_update_steps_per_epoch)

    if accelerator.is_main_process:
        accelerator.init_trackers("text2image-fine-tune", config=vars(config))

    total_batch_size = config.train_batch_size * accelerator.num_processes * config.gradient_accumulation_steps

    log_init(logger, len(train_dataset), config, total_batch_size)
    global_step = 0
    first_epoch = 0

    # Potentially load in the weights and states from a previous save
    if config.resume_from_checkpoint:
        if config.resume_from_checkpoint:
            path = os.path.basename(config.resume_from_checkpoint)

        if path is None:
            accelerator.print(
                f"Checkpoint '{config.resume_from_checkpoint}' does not exist. Starting a new training run."
            )
            config.resume_from_checkpoint = None
            initial_global_step = 0
        else:
            accelerator.print(f"Resuming from checkpoint {path}")
            accelerator.load_state(os.path.join(config.output_dir, path))
            global_step = int(path.split("-")[1])

            initial_global_step = global_step
            first_epoch = global_step // num_update_steps_per_epoch
    else:
        initial_global_step = 0

    progress_bar = tqdm(range(0, config.max_train_steps),initial=initial_global_step,desc="Steps",disable=not accelerator.is_local_main_process)

    # ------------TRAINING LOOP------------
    for epoch in range(first_epoch, config.num_train_epochs):
        unet.train()
        train_loss = 0.0
        for _, batch in enumerate(train_dataloader):
            with accelerator.accumulate(unet):

                # Convert images to latent space
                latents = vae.encode(batch["pixel_values"].to(dtype=weight_dtype)).latent_dist.sample()
                latents = latents * vae.config.scaling_factor

                # Sample noise that we'll add to the latents
                noise = torch.randn_like(latents)

                bsz = latents.shape[0]
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
                timesteps = timesteps.long()

                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
                encoder_hidden_states = text_encoder(batch["input_ids"], return_dict=False)[0]

                if noise_scheduler.config.prediction_type == "epsilon":
                    target = noise
                elif noise_scheduler.config.prediction_type == "v_prediction":
                    target = noise_scheduler.get_velocity(latents, noise, timesteps)
                else:
                    raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")

                # Predict the noise residual and compute loss
                model_pred = unet(noisy_latents, timesteps, encoder_hidden_states, return_dict=False)[0]

                loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")

                # Gather the losses across all processes for logging (if we use distributed training).
                avg_loss = accelerator.gather(loss.repeat(config.train_batch_size)).mean()
                train_loss += avg_loss.item() / config.gradient_accumulation_steps

                # Backpropagate
                accelerator.backward(loss)
                if accelerator.sync_gradients:
                    params_to_clip = lora_layers
                    accelerator.clip_grad_norm_(params_to_clip, config.max_grad_norm)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            # Checks if the accelerator has performed an optimization step behind the scenes
            if accelerator.sync_gradients:
                progress_bar.update(1)
                global_step += 1
                accelerator.log({"train_loss": train_loss}, step=global_step)
                train_loss = 0.0

                if global_step % config.checkpointing_steps == 0:
                    if accelerator.is_main_process:
                        save_path = os.path.join(config.output_dir, f"checkpoint-{global_step}")
                        accelerator.save_state(save_path)
                        logger.info(f"Saved state to {save_path}")

            logs = {"step_loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
            progress_bar.set_postfix(**logs)

            if global_step >= config.max_train_steps:
                break

        if accelerator.is_main_process:
            if config.validation_prompt is not None and epoch % config.validation_epochs == 0:
                # create pipeline
                pipeline = DiffusionPipeline.from_pretrained(
                    config.pretrained_model_name_or_path,
                    unet=unwrap_model(unet),
                    torch_dtype=weight_dtype,
                )
                images = log_validation(logger, pipeline, config, accelerator, epoch)

                del pipeline
                torch.cuda.empty_cache()

    # Save the lora layers
    accelerator.wait_for_everyone()
    if accelerator.is_main_process:
        unet = unet.to(torch.float32)

        unwrapped_unet = unwrap_model(unet)
        unet_lora_state_dict = convert_state_dict_to_diffusers(get_peft_model_state_dict(unwrapped_unet))
        StableDiffusionPipeline.save_lora_weights(
            save_directory=config.output_dir,
            unet_lora_layers=unet_lora_state_dict,
            safe_serialization=True,
        )

        if config.push_to_hub:
            save_model_card(
                repo_id,
                base_model=config.pretrained_model_name_or_path,
                repo_folder=config.output_dir,
            )
            upload_folder(
                repo_id=repo_id,
                folder_path=config.output_dir,
                commit_message="End of training",
                ignore_patterns=["step_*", "epoch_*"],
            )

    accelerator.end_training()

ImportError: This example requires a source install from HuggingFace diffusers (see `https://huggingface.co/docs/diffusers/installation#install-from-source`), but the version found is 0.35.2.


In [ ]:
train()

INFO:__main__:Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: bf16

{'sample_max_value', 'timestep_spacing', 'prediction_type', 'thresholding', 'variance_type', 'rescale_betas_zero_snr', 'clip_sample_range', 'dynamic_thresholding_ratio'} was not found in config. Values will be initialized to default values.
{'force_upcast', 'mid_block_add_attention', 'latents_std', 'use_quant_conv', 'shift_factor', 'latents_mean', 'scaling_factor', 'use_post_quant_conv'} was not found in config. Values will be initialized to default values.
All model checkpoint weights were used when initializing AutoencoderKL.

All the weights of AutoencoderKL were initialized from the model checkpoint at stable-diffusion-v1-5/stable-diffusion-v1-5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use AutoencoderKL for predictions without further training.
{'addition_time_embed_dim', 

INFO:__main__:***** Training *****
INFO:__main__:  Num examples = 2749
INFO:__main__:  Num Epochs = 22
INFO:__main__:  Instantaneous batch size per device = 8
INFO:__main__:  Total train batch size (w. parallel, distributed & accumulation) = 8
INFO:__main__:  Gradient Accumulation steps = 1
INFO:__main__:  Total optimization steps = 7500


Steps:   0%|          | 0/7500 [00:00<?, ?it/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

...

Loaded text_encoder as CLIPTextModel from `text_encoder` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
Loaded tokenizer as CLIPTokenizer from `tokenizer` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
{'timestep_spacing', 'prediction_type'} was not found in config. Values will be initialized to default values.
Loaded scheduler as PNDMScheduler from `scheduler` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
Instantiating AutoencoderKL model under default dtype torch.bfloat16.
{'force_upcast', 'mid_block_add_attention', 'latents_std', 'use_quant_conv', 'shift_factor', 'latents_mean', 'scaling_factor', 'use_post_quant_conv'} was not found in config. Values will be initialized to default values.
All model checkpoint weights were used when initializing AutoencoderKL.

All the weights of AutoencoderKL were initialized from the model checkpoint at /root/.cache/huggingface/hub/models--stable-diffusion-v1-5--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d22

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loaded text_encoder as CLIPTextModel from `text_encoder` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
Loaded tokenizer as CLIPTokenizer from `tokenizer` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
{'timestep_spacing', 'prediction_type'} was not found in config. Values will be initialized to default values.
Loaded scheduler as PNDMScheduler from `scheduler` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
Instantiating AutoencoderKL model under default dtype torch.bfloat16.
{'force_upcast', 'mid_block_add_attention', 'latents_std', 'use_quant_conv', 'shift_factor', 'latents_mean', 'scaling_factor', 'use_post_quant_conv'} was not found in config. Values will be initialized to default values.
All model checkpoint weights were used when initializing AutoencoderKL.

All the weights of AutoencoderKL were initialized from the model checkpoint at /root/.cache/huggingface/hub/models--stable-diffusion-v1-5--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d22

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loaded text_encoder as CLIPTextModel from `text_encoder` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
Loaded tokenizer as CLIPTokenizer from `tokenizer` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
{'timestep_spacing', 'prediction_type'} was not found in config. Values will be initialized to default values.
Loaded scheduler as PNDMScheduler from `scheduler` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
Instantiating AutoencoderKL model under default dtype torch.bfloat16.
{'force_upcast', 'mid_block_add_attention', 'latents_std', 'use_quant_conv', 'shift_factor', 'latents_mean', 'scaling_factor', 'use_post_quant_conv'} was not found in config. Values will be initialized to default values.
All model checkpoint weights were used when initializing AutoencoderKL.

All the weights of AutoencoderKL were initialized from the model checkpoint at /root/.cache/huggingface/hub/models--stable-diffusion-v1-5--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d22

In [ ]:
config = TrainingConfig()
repo_id = create_repo(
    repo_id="SurAyush/StableEmote_LoRA",
    exist_ok=True,
    token=config.hub_token,
).repo_id
save_model_card(
    repo_id,
    base_model=config.pretrained_model_name_or_path,
    repo_folder=config.output_dir,
)
upload_folder(
    repo_id=repo_id,
    folder_path=config.output_dir,
    commit_message="End of training",
    ignore_patterns=["step_*", "epoch_*"],
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...kpoint-1000/optimizer.bin:  10%|#         |  342kB / 3.40MB            

  ..._lora_weights.safetensors:  10%|#         |  164kB / 1.63MB            

  ...-1000/random_states_0.pkl:  10%|#         | 1.49kB / 14.8kB            

...

  ...-1500/random_states_0.pkl:  10%|#         | 1.49kB / 14.8kB            

  ...kpoint-1500/scheduler.bin:  10%|9         |   140B / 1.40kB            

  ...kpoint-2000/optimizer.bin:  10%|#         |  342kB / 3.40MB            

  ..._lora_weights.safetensors:  10%|#         |  164kB / 1.63MB            

CommitInfo(commit_url='https://huggingface.co/SurAyush/StableEmote_LoRA/commit/08793b0bb4f90ce9e4d6758c803ee3b585a79692', commit_message='End of training', commit_description='', oid='08793b0bb4f90ce9e4d6758c803ee3b585a79692', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SurAyush/StableEmote_LoRA', endpoint='https://huggingface.co', repo_type='model', repo_id='SurAyush/StableEmote_LoRA'), pr_revision=None, pr_num=None)